In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyrootutils
import hydra
import torch

root = pyrootutils.setup_root(search_from="./", indicator=[".git", "pyproject.toml"], pythonpath=True, dotenv=True)


In [3]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize("../configs/model/", version_base="1.2")
model_config = hydra.compose("mae3d_classification.yaml")

/home/users/shubham.kumar/miniconda3/envs/qct_deep/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'mae3d_classification.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [4]:
model_config['scheduler']['epochs'] = 300

In [5]:
model_config

{'optimizer': {'_target_': 'torch.optim.SGD', '_partial_': True, 'lr': 0.1, 'weight_decay': 0.0001, 'momentum': 0.9}, 'scheduler': {'_target_': 'torch.optim.lr_scheduler.OneCycleLR', '_partial_': True, 'max_lr': 0.5, 'epochs': 300}, 'model': {'net': {'_target_': 'src.common.nn_modules.nets.mae.mae3d_classification.ViTMAEForImageClassification', 'model_ckpt': '/data_nas5/qer/shubham/mae3d_checkpoints/fpr_mae/runs/2023-12-20_20-17-17/checkpoints/epoch=75_step=8436_val_loss=0.01.ckpt', 'model': {'_target_': 'src.common.nn_modules.nets.mae.mae3d.MAE3D', 'pos_embed_type': 'sincos', 'mask_ratio': 0.7, 'input_size': [48, 48, 16], 'patch_size': [6, 6, 4], 'in_chans': 1, 'encoder_embed_dim': 576, 'encoder_depth': 4, 'encoder_num_heads': 12, 'decoder_embed_dim': 252, 'decoder_depth': 3, 'decoder_num_heads': 12, 'encoder_drop_rate': 0.15, 'attn_drop_rate': 0.1}}}, '_target_': 'src.models.combined_module.CombinedLitModule', 'activation': {'_target_': 'torch.nn.Softmax', 'dim': 1}, 'metric': {'_tar

In [6]:
model_config['model']['net']

{'_target_': 'src.common.nn_modules.nets.mae.mae3d_classification.ViTMAEForImageClassification', 'model_ckpt': '/data_nas5/qer/shubham/mae3d_checkpoints/fpr_mae/runs/2023-12-20_20-17-17/checkpoints/epoch=75_step=8436_val_loss=0.01.ckpt', 'model': {'_target_': 'src.common.nn_modules.nets.mae.mae3d.MAE3D', 'pos_embed_type': 'sincos', 'mask_ratio': 0.7, 'input_size': [48, 48, 16], 'patch_size': [6, 6, 4], 'in_chans': 1, 'encoder_embed_dim': 576, 'encoder_depth': 4, 'encoder_num_heads': 12, 'decoder_embed_dim': 252, 'decoder_depth': 3, 'decoder_num_heads': 12, 'encoder_drop_rate': 0.15, 'attn_drop_rate': 0.1}}

In [7]:
model = hydra.utils.instantiate(model_config['model']['net'])

/home/users/shubham.kumar/miniconda3/envs/qct_deep/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
ckpt_path_v1 = "/data_nas5/qer/shubham/fpr_sl_checkpoints/fpr_sl/runs/2023-12-11_20-07-27/checkpoints/epoch=28_step=667_val_loss=0.09.ckpt"



In [9]:
ckpt_path_v2 = "/data_nas5/qer/shubham/mae3d_clf/fpr_mae3d_clf/runs/2023-12-19_08-46-36/checkpoints/epoch=140_step=3243_val_loss=0.09.ckpt"


In [8]:
ckpt_path = "/data_nas5/qer/shubham/mae3d_clf/fpr_mae3d_clf/runs/2023-12-21_13-14-08/checkpoints/last.ckpt"

In [9]:
from src.common.nn_modules.nets.segmentation.multitaskfusion import modify_model_state_dict

In [10]:
k= torch.load(ckpt_path, map_location = "cpu")
state_dict = k['state_dict']
new_state_dict = modify_model_state_dict(state_dict , "net." , "")

model.load_state_dict(new_state_dict)

<All keys matched successfully>

In [11]:
!nvidia-smi

Mon Jan  8 10:25:22 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 37%   37C    P8    30W / 350W |  15862MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
| 54%   

In [12]:
model = model.to("cuda:2")

In [13]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize("../configs/datamodule/", version_base="1.2")
data_module = hydra.compose("fpr_sl_mae_datamodule.yaml")

In [14]:
data_module

{'transforms': {'val_transforms': {'_target_': 'monai.transforms.Compose', 'transforms': [{'_target_': 'src.common.transforms.monai_extras.LoadSafeTensor', 'image_key': 'image'}, {'_target_': 'src.common.transforms.monai_extras.LoadSafeTensor_FPR_SL', 'image_key': 'image'}, {'_target_': 'src.common.transforms.monai_extras.Resize3D', 'image_key': 'image', 'spatial_size': [16, 48, 48]}, {'_target_': 'src.common.transforms.monai_extras.ApplyWindowsChannelWise', 'keys': ['image'], 'windows': ['LungA']}]}, 'train_transforms': {'_target_': 'monai.transforms.Compose', 'transforms': [{'_target_': 'src.common.transforms.monai_extras.LoadSafeTensor', 'image_key': 'image'}, {'_target_': 'src.common.transforms.monai_extras.LoadSafeTensor_FPR_SL', 'image_key': 'image'}, {'_target_': 'src.common.transforms.monai_extras.Resize3D', 'image_key': 'image', 'spatial_size': [16, 48, 48]}, {'_target_': 'src.common.transforms.monai_extras.ApplyWindowsChannelWise', 'keys': ['image'], 'windows': ['LungA']}, {'

In [17]:
datamodule = hydra.utils.instantiate(data_module)

2024-01-08 10:27:48.162 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:_load_data:127 - wcg
2024-01-08 10:27:48.166 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:_load_data:127 - qidw
2024-01-08 10:27:48.167 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:_load_data:127 - qure_external_v1
2024-01-08 10:27:48.169 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:_load_data:156 - positive: 0 , negative: 0
2024-01-08 10:27:48.413 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:_load_data_neg:16 - lidc
2024-01-08 10:27:48.780 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:_load_data_neg:16 - nlst
2024-01-08 10:27:49.167 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:_load_data_pos:37 - lidc
2024-01-08 10:27:49.320 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:_load_data_pos:37 - nlst
2024-01-08 10:27:49.945 | DEBUG    | src.datamodules.load_data.fpr_sl_mae:fpr_sl_data_loader:108 - train : positive: 6403 , negative: 30000
2024-01-08 10:27:49.967 | DEBUG    | src.datamod

In [16]:
train_ds = datamodule.train_ds
val_ds = datamodule.val_ds
test_ds = datamodule.test_ds

In [18]:
test_ds[0]

{'datapath': '/cache/fast_data_nas8/qct/shubham/fpr_cache_48_48_16/safetensors_cache/1.3.6.1.4.1.14519.5.2.1.6279.6001.119806527488108718706404165837.safetensors',
 'label': 0,
 'source': 'lidc_luna',
 'data_type': 1,
 'image': tensor([[[[0.2962, 0.3013, 0.3100,  ..., 0.5088, 0.4313, 0.3125],
           [0.2862, 0.3088, 0.3338,  ..., 0.5700, 0.5587, 0.4338],
           [0.3475, 0.4212, 0.4212,  ..., 0.6938, 0.6637, 0.5337],
           ...,
           [0.3313, 0.3225, 0.3113,  ..., 0.8888, 0.8675, 0.7650],
           [0.3200, 0.3288, 0.3212,  ..., 0.7962, 0.7775, 0.7150],
           [0.3113, 0.3237, 0.3262,  ..., 0.5875, 0.5925, 0.5788]],
 
          [[0.2937, 0.3125, 0.3200,  ..., 0.5188, 0.4175, 0.3275],
           [0.2912, 0.3388, 0.3663,  ..., 0.6575, 0.6137, 0.5038],
           [0.3825, 0.4750, 0.4762,  ..., 0.8150, 0.7763, 0.6550],
           ...,
           [0.3425, 0.3363, 0.3262,  ..., 0.8888, 0.9050, 0.8612],
           [0.3487, 0.3562, 0.3325,  ..., 0.8112, 0.8462, 0.8400],
 

In [51]:
import pandas as pd
df_fpr_qidw = pd.read_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/fpr_wcg.csv")

In [52]:
import os

In [54]:
qidw_dict = torch.load("/cache/fast_data_nas72/qct/data_governance/series_dicts/wcg_fda_qct.pt")

In [55]:
map_study_id_series_id = {}

li_series_ids = os.listdir("/cache/fast_data_nas8/qct/shubham/fpr_cache_wcg")
li_series_ids = [item[:-len(".safetensors")] for item in li_series_ids]


In [56]:
for suid in li_series_ids :
    study_uid =  qidw_dict[suid]["StudyInstanceUID"]
    map_study_id_series_id[study_uid] = suid

In [57]:
for idx in df_fpr_qidw.index :
    study_uid = df_fpr_qidw.loc[idx, "SeriesUID"]
    df_fpr_qidw.loc[idx, "SeriesUID"] =  map_study_id_series_id[study_uid]

KeyError: '1.3.6.1.4.1.55648.6361110350220339743275615199544204'

In [50]:
df_fpr_qidw.to_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/fpr_qidw.csv",index =False)

In [48]:
for i in df_fpr_qidw.index :
    suid = df_fpr_qidw.loc[i, "SeriesUID"]
    print(suid)
    path = os.path.join("/cache/fast_data_nas8/qct/shubham/fpr_cache_qidw/" , f"{suid}.safetensors")
    if os.path.exists(path):
        print(path)

1.3.6.1.4.1.9328.50.1.334215469883390884437229836705492234507
/cache/fast_data_nas8/qct/shubham/fpr_cache_qidw/1.3.6.1.4.1.9328.50.1.334215469883390884437229836705492234507.safetensors
1.3.6.1.4.1.9328.50.1.116324576791139455082825091960790224867
/cache/fast_data_nas8/qct/shubham/fpr_cache_qidw/1.3.6.1.4.1.9328.50.1.116324576791139455082825091960790224867.safetensors
1.3.6.1.4.1.9328.50.1.37531487280017801124765643732079524443
/cache/fast_data_nas8/qct/shubham/fpr_cache_qidw/1.3.6.1.4.1.9328.50.1.37531487280017801124765643732079524443.safetensors
1.3.6.1.4.1.9328.50.1.177903673228104341213224928761701786693
/cache/fast_data_nas8/qct/shubham/fpr_cache_qidw/1.3.6.1.4.1.9328.50.1.177903673228104341213224928761701786693.safetensors
1.3.6.1.4.1.9328.50.1.70949788529300490780576543460245043945
/cache/fast_data_nas8/qct/shubham/fpr_cache_qidw/1.3.6.1.4.1.9328.50.1.70949788529300490780576543460245043945.safetensors
1.3.6.1.4.1.9328.50.1.112793496676844175431872842334447612042
/cache/fast_data_

In [17]:
from torch.utils.data import DataLoader

In [20]:
dataloader = DataLoader(
    train_ds,
    batch_size= 16,
    shuffle=False,
    num_workers=16,
    drop_last=True,
    pin_memory=True,
)


In [21]:
activation = torch.nn.Softmax(dim = 1)

In [22]:
result = []
labels = []
source = []
for batch in dataloader :
    image = batch["image"].to("cuda:2")
    output = activation(model.forward(image))[:,1].detach().cpu()
    result.append(output)
    labels.append(batch["label"])
    source.append(batch["source"])

In [23]:
flattened_list = [item for sublist in source for item in sublist]


In [24]:
set(flattened_list)

{'lidc_luna', 'nlst_pm_t0'}

In [34]:
indices_val1 = [index for index, element in enumerate(my_list) if element == val1]
indices_val2 = [index for index, element in enumerate(my_list) if element == val2]
indices_val3 = [index for index, element in enumerate(my_list) if element == val3]
indices_val4 = [index for index, element in enumerate(my_list) if element == val4]


['lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc_luna',
 'lidc

In [32]:
final_out = torch.concat(result).numpy()
final_labels = torch.concat(labels).squeeze().numpy()


In [32]:
import numpy as np
np.unique(final_labels)

array([1])

In [33]:
final_out.shape

(4704,)

In [34]:
len(final_out[final_out > 0.5])

3510

In [25]:
len(final_out[final_out < 0.5])

254

In [49]:
specificity = 116381/117952

In [50]:
specificity

0.9866810227889311

In [26]:
sens = 578/832

In [38]:
sens = len(final_out[final_out > 0.3])/final_out.shape[0]
2

0.7772108843537415

In [58]:
specificity = len(final_out[final_out < 0.5])/final_out.shape[0]
specificity

0.9849509947337625

In [66]:
specificity = len(final_out[final_out < 0.5])/final_out.shape[0]
specificity

0.9744552526232789

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("/cache/fast_data_nas8/qct/shubham/luna16/candidates.csv")

In [6]:
df2 = pd.read_csv("/cache/fast_data_nas8/qct/shubham/luna16/annotations.csv")

In [7]:
df2

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150
...,...,...,...,...,...
1181,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-160.856298,-28.560349,-269.168728,5.053694
1182,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-102.189570,-73.865766,-220.536241,4.556101
1183,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-37.535409,64.041949,-127.687101,4.357368
1184,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,43.196112,74.438486,-200.523314,4.277203


In [8]:
df3 = pd.read_csv("/cache/fast_data_nas8/qct/shubham/luna16/candidates_V2/candidates_V2.csv")

In [36]:
df3

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0
...,...,...,...,...,...
754970,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-33.400000,-64.200000,-115.560000,0
754971,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,56.236359,70.352400,-203.446236,0
754972,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-97.104221,55.738289,-203.879785,0
754973,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-65.470000,59.670000,-136.370000,0


In [42]:
df3[df3["class"] == 1]

,seriesuid,coordX,coordY,coordZ,class
436,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,104.083933,-211.755826,-227.017987,1
1009,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.982091,-175.176790,-298.510193,1
2053,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.974375,-141.066875,876.777280,1
3633,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,1.790000,166.340000,-408.880000,1
3707,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,1.859783,172.221534,-405.366447,1
...,...,...,...,...,...
754090,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-29.720000,24.620000,-156.010000,1
754622,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,42.895122,74.251037,-200.625978,1
754755,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-28.685000,32.690000,-164.835000,1
754947,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-23.719297,31.714489,-156.172233,1


In [12]:
df[df["class"] == 1]

,seriesuid,coordX,coordY,coordZ,class
13,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,104.164804,-211.685591,-227.011364,1
78,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.940000,-175.040000,-297.870000,1
1303,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,70.190000,-140.930000,877.680000,1
3050,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,89.320000,190.840000,-516.820000,1
3052,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,94.930000,153.030000,-429.910000,1
...,...,...,...,...,...
548674,1.3.6.1.4.1.14519.5.2.1.6279.6001.979083010707...,-102.840925,57.880915,-124.815520,1
550171,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-161.330000,-29.090000,-269.290000,1
550334,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-23.719297,31.714489,-156.172233,1
550810,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,43.129934,74.326554,-200.283918,1


In [13]:
df4 = pd.read_csv("/cache/fast_data_nas8/qct/shubham/luna16/evaluationScript/evaluationScript/annotations/annotations.csv")


In [15]:
df4

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150
...,...,...,...,...,...
1181,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-160.856298,-28.560349,-269.168728,5.053694
1182,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-102.189570,-73.865766,-220.536241,4.556101
1183,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-37.535409,64.041949,-127.687101,4.357368
1184,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,43.196112,74.438486,-200.523314,4.277203


In [16]:
df5 = pd.read_csv("/cache/fast_data_nas8/qct/shubham/luna16/evaluationScript/evaluationScript/annotations/annotations_excluded.csv")


In [20]:
import numpy as np
np.unique(df5.diameter_mm.values)

array([-1.        ,  2.03222276,  2.15282627, ..., 32.28626813,
       32.42732783, 38.14769884])

In [35]:
df5[(df5["diameter_mm"] != -1)]

,seriesuid,coordX,coordY,coordZ,diameter_mm
12,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.780294,-211.860024,-227.042500,4.630484
13,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.491874,-175.253974,-298.570757,5.441647
35,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.590963,-211.880166,-227.035000,4.852332
36,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.513446,-175.244635,-298.557304,5.422923
58,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.516454,-175.157075,-298.544186,4.970728
...,...,...,...,...,...
35161,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-160.874865,-28.510371,-269.362222,4.165758
35174,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-21.851333,33.018881,-156.386735,24.568500
35175,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,43.314061,74.521844,-200.526077,4.083998
35183,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-21.938878,32.923503,-156.711266,24.970845


In [52]:
df5

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-131.896494,-155.056702,-317.80,-1.0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,80.798736,-109.295001,-123.40,-1.0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-62.287146,-55.154396,-283.60,-1.0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,83.376859,-58.377052,-236.80,-1.0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-41.017623,-60.310645,-240.40,-1.0
...,...,...,...,...,...
35187,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-45.043687,57.018889,-99.11,-1.0
35188,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-31.567113,20.690733,-15.36,-1.0
35189,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-38.598369,39.440749,-179.11,-1.0
35190,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,29.370439,16.589167,15.89,-1.0


In [86]:
df5[df5['diameter_mm'] > 0]

,seriesuid,coordX,coordY,coordZ,diameter_mm
12,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.780294,-211.860024,-227.042500,4.630484
13,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.491874,-175.253974,-298.570757,5.441647
35,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.590963,-211.880166,-227.035000,4.852332
36,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.513446,-175.244635,-298.557304,5.422923
58,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.516454,-175.157075,-298.544186,4.970728
...,...,...,...,...,...
35161,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-160.874865,-28.510371,-269.362222,4.165758
35174,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-21.851333,33.018881,-156.386735,24.568500
35175,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,43.314061,74.521844,-200.526077,4.083998
35183,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-21.938878,32.923503,-156.711266,24.970845


In [50]:
from scipy.spatial.distance import cdist
import numpy as np

candidates_df = df3
annotations_df = df5

# Filtering candidates and annotations that represent nodules
nodule_candidates = candidates_df[candidates_df['class'] == 1]
nodule_annotations = annotations_df[annotations_df['diameter_mm'] > 0]

# Merging both dataframes based on seriesuid
merged_df = nodule_candidates.merge(nodule_annotations, on='seriesuid', suffixes=('_cand', '_anno'))

# Function to calculate Euclidean distance
def calculate_distance(row):
    point1 = np.array([row['coordX_cand'], row['coordY_cand'], row['coordZ_cand']])
    point2 = np.array([row['coordX_anno'], row['coordY_anno'], row['coordZ_anno']])
    return np.linalg.norm(point1 - point2)

# Apply the function to calculate distances
merged_df['distance'] = merged_df.apply(calculate_distance, axis=1)

# Assuming a threshold (can be adjusted)
distance_threshold = 5  # this can be changed based on what is considered "very close"

# Filter rows where the distance is below the threshold
close_points_df = merged_df[merged_df['distance'] <= distance_threshold]

close_points_df.head()


,seriesuid,coordX_cand,coordY_cand,coordZ_cand,class,coordX_anno,coordY_anno,coordZ_anno,diameter_mm,distance
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,104.083933,-211.755826,-227.017987,1,103.780294,-211.860024,-227.042500,4.630484,0.321955
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,104.083933,-211.755826,-227.017987,1,103.590963,-211.880166,-227.035000,4.852332,0.508694
6,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.982091,-175.176790,-298.510193,1,-128.491874,-175.253974,-298.570757,5.441647,0.499938
8,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.982091,-175.176790,-298.510193,1,-128.513446,-175.244635,-298.557304,5.422923,0.475869
9,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.982091,-175.176790,-298.510193,1,-128.516454,-175.157075,-298.544186,4.970728,0.467293


In [84]:
close_points_df

,seriesuid,coordX_cand,coordY_cand,coordZ_cand,class,coordX_anno,coordY_anno,coordZ_anno,diameter_mm,distance
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,104.083933,-211.755826,-227.017987,1,103.780294,-211.860024,-227.042500,4.630484,0.321955
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,104.083933,-211.755826,-227.017987,1,103.590963,-211.880166,-227.035000,4.852332,0.508694
6,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.982091,-175.176790,-298.510193,1,-128.491874,-175.253974,-298.570757,5.441647,0.499938
8,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.982091,-175.176790,-298.510193,1,-128.513446,-175.244635,-298.557304,5.422923,0.475869
9,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.982091,-175.176790,-298.510193,1,-128.516454,-175.157075,-298.544186,4.970728,0.467293
...,...,...,...,...,...,...,...,...,...,...
15440,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-161.010000,-28.890000,-269.370000,1,-160.874865,-28.510371,-269.362222,4.165758,0.403038
15446,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,42.895122,74.251037,-200.625978,1,43.314061,74.521844,-200.526077,4.083998,0.508750
15448,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,42.895122,74.251037,-200.625978,1,43.317685,74.510579,-200.544426,3.683960,0.502565
15453,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-23.719297,31.714489,-156.172233,1,-21.851333,33.018881,-156.386735,24.568500,2.288392


In [73]:
bb = close_points_df[["seriesuid" , "coordX_anno", "coordY_anno","coordZ_anno","diameter_mm"]]

In [76]:
bb = bb.rename(columns={"coordX_anno": "coordX" , "coordY_anno":"coordY", "coordZ_anno":"coordZ"})

In [87]:
aa

,seriesuid,coordX,coordY,coordZ,diameter_mm
12,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.780294,-211.860024,-227.042500,4.630484
13,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.491874,-175.253974,-298.570757,5.441647
35,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.590963,-211.880166,-227.035000,4.852332
36,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.513446,-175.244635,-298.557304,5.422923
58,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.516454,-175.157075,-298.544186,4.970728
...,...,...,...,...,...
35161,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-160.874865,-28.510371,-269.362222,4.165758
35174,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-21.851333,33.018881,-156.386735,24.568500
35175,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,43.314061,74.521844,-200.526077,4.083998
35183,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-21.938878,32.923503,-156.711266,24.970845


In [77]:
aa = annotations_df[annotations_df["diameter_mm"]!=-1]

These are those rows/nodule_coordinates of annotations_excluded.csv which are not present in candidate_V2.csv

In [88]:
final_annotations_excluded =  pd.concat([aa ,bb]).drop_duplicates(keep=False)
final_annotations_excluded

,seriesuid,coordX,coordY,coordZ,diameter_mm
75,1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840...,43.837410,78.367618,-60.951290,3.629197
79,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,-68.467672,-82.596983,866.906897,3.389565
80,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,19.575837,-213.594633,937.097132,4.018189
110,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,19.584936,-213.645833,937.223077,3.741389
279,1.3.6.1.4.1.14519.5.2.1.6279.6001.100953483028...,-70.423396,56.883648,-200.028002,6.303262
...,...,...,...,...,...
35053,1.3.6.1.4.1.14519.5.2.1.6279.6001.986011151772...,-26.751294,76.669673,-90.558065,5.087544
35055,1.3.6.1.4.1.14519.5.2.1.6279.6001.986011151772...,-26.875589,76.628847,-90.516129,5.269864
35071,1.3.6.1.4.1.14519.5.2.1.6279.6001.986011151772...,103.066850,35.155813,-169.271277,6.930138
35125,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,167.375236,38.276147,-196.756300,3.901359
